### Problem 1 :

#### Definition of the MLP class:

In [ ]:
import numpy as np
import pickle

# MLP class :

# The model implemented as follows :
# Each layers is represented by a b vector (biases) and a W matrix (weights)
# These are referenced by the weights dictionary. The format is :
# self.weights[f"X{n}"] where X = b, W
# NB : In our implementation, these matrices are transposed compared to the class notations

class NN(object):
    
    def __init__(self,
                 hidden_dims=(1024, 2048), # dimensions of each hidden layers
                 n_hidden=2, # number of hidden layers
                 mode='train', # current mode : train/test
                 datapath=None, # path where to find the .pkl file
                 model_path=None, # path where to save/load the model 
                 epsilon = 1e-6, # for cross entropy calculus stability : log(x) = log(epsilon) if x < epsilon
                 lr = 1e-1, # learning rate
                 n_epochs = 1000, # max number of epochs
                 batch_size = 1000): # batch size for training
        
        assert len(hidden_dims) == n_hidden, "Hidden dims mismatch!"
        
        self.hidden_dims = hidden_dims
        self.n_hidden = n_hidden
        self.mode = mode
        self.datapath = datapath
        self.model_path = model_path
        self.epsilon = epsilon
        self.lr = lr
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        
        # train, validation and test sets :
        #self.tr, self.va, self.te = np.load(open(datapath, "rb"))
        u = pickle._Unpickler(open(datapath, 'rb'))
        u.encoding = 'latin1'
        self.tr, self.va, self.te = u.load()

    def initialize_weights(self, dims, method):
        """
        Initializes the weights and biases according to the specified method
        Parameters :
        - dims: (list of two integers) - the size of input/output layers
        - method: (string) - initializes the weight matrices
            -> "zero" for a Zero initialisation of the weights
            -> "normal" for a Normal initialisation of the weights
            -> "glorot" for a Uniform initialisation of the weights
        :return: None
        """
        if self.mode == "train":
            
            self.weights = {}
            all_dims = [dims[0]] + list(self.hidden_dims) + [dims[1]]
            print("Layers dimensions are : ", all_dims)
            
            for layer_n in range(1, self.n_hidden + 2):
                if method == "zero":
                    self.weights[f"W{layer_n}"] = np.zeros(shape=(all_dims[layer_n - 1],all_dims[layer_n]))
                elif method == "normal":
                    self.weights[f"W{layer_n}"] = np.random.normal(loc=0.0, scale=1.0, size=(all_dims[layer_n - 1],all_dims[layer_n]))
                elif method == "glorot":
                    b = np.sqrt(6.0/(all_dims[layer_n]+all_dims[layer_n-1]))
                    self.weights[f"W{layer_n}"] = np.random.uniform(low=-1*b, high=b, size=(all_dims[layer_n - 1],all_dims[layer_n]))
                else:
                    raise Exception("The provided name for the initialization method is invalid.")
                print("Initialized W",layer_n,":\n",self.weights[f"W{layer_n}"])
                self.weights[f"b{layer_n}"] = np.zeros((1, all_dims[layer_n]))  # np.random.rand(1, all_dims[layer_n])
                
        elif self.mode == "test":
            pass
        else:
            raise Exception("Unknown Mode!")

    def activation(self, input, prime=False): # Prime for Heavyside, else ReLu
        if prime:
            return input > 0
        return np.maximum(0, input)

    def loss(self, prediction, labels):  # Computes the cross entropy
        prediction[np.where(prediction < self.epsilon)] = self.epsilon
        prediction[np.where(prediction > 1 - self.epsilon)] = 1 - self.epsilon
        return -1 * np.sum(labels * np.log(prediction) + (1-labels) * np.log(1-prediction)) / prediction.shape[0]

    def softmax(self, input):  # Computes the softmax of the input
        Z = np.exp(input - np.max(input)) # softmax(x+C) = softmax(x) (stability)
        return Z / np.sum(Z, axis=1, keepdims=True)
    
    def forward(self, input):  # Forward propagation : computes the outputs (cache) from the input
        # TODO : description
        cache = {"H0": input}
        for layer in range(1, self.n_hidden + 1):
            cache[f"A{layer}"] = cache[f"H{layer-1}"] @ self.weights[f"W{layer}"] + self.weights[f"b{layer}"]
            cache[f"H{layer}"] = self.activation(cache[f"A{layer}"])
        layer = self.n_hidden + 1
        cache[f"A{layer}"] = cache[f"H{layer-1}"] @ self.weights[f"W{layer}"] + self.weights[f"b{layer}"]
        cache[f"H{layer}"] = self.softmax(cache[f"A{layer}"]) # softmax on last layer
        return cache

    def backward(self, cache, labels):  # Backward propagation : computes the gradients from the outputs (cache)
        # TODO: description, gradient of the loss function?
        output = cache[f"H{self.n_hidden+1}"]
        grads = {
            f"dA{self.n_hidden+1}": - (labels - output),
            # f"dA{self.n_hidden+1}": - (labels - self.loss(output, labels))
        }
        for layer in range(self.n_hidden + 1, 0, -1):
            # print(f"Shape dA=", grads[f"dA{layer}"].shape)
            # print(f"Shape H=", cache[f"H{layer-1}"].shape)

            grads[f"dW{layer}"] = cache[f"H{layer-1}"].T @ grads[f"dA{layer}"]
            grads[f"db{layer}"] = grads[f"dA{layer}"]

            if layer > 1:
                grads[f"dH{layer-1}"] = grads[f"dA{layer}"] @ self.weights[f"W{layer}"].T
                grads[f"dA{layer-1}"] = grads[f"dH{layer-1}"] * self.activation(cache[f"A{layer-1}"], prime=True)
                # print(f"Shape dA=", grads[f"dA{layer-1}"].shape)
        return grads

    def update(self, grads):  #
        # print(grads.keys())
        for layer in range(1, self.n_hidden + 1):
            # print(grads[f"dW{layer}"].shape,self.weights[f"W{layer}"].shape)
            self.weights[f"W{layer}"] = self.weights[f"W{layer}"] - self.lr * grads[f"dW{layer}"] / self.batch_size # why division ?
            #self.weights[f"b{layer}"] = self.weights[f"b{layer}"] - self.lr * grads[f"db{layer}"] # / self.batch_size

    def train(self, initializationMethod):
        X_train, y_train = self.tr
        y_onehot = np.eye(np.max(y_train) - np.min(y_train) + 1)[y_train]
        # print(y_train.shape,y_onehot.shape)
        dims = [X_train.shape[1], y_onehot.shape[1]]
        self.initialize_weights(dims, initializationMethod)

        n_batches = int(np.ceil(X_train.shape[0] / self.batch_size))

        for epoch in range(self.n_epochs):
            predictedY = np.zeros_like(y_train)
            trainLoss = 0
            for batch in range(n_batches):
                minibatchX = X_train[self.batch_size * batch:self.batch_size * (batch + 1), :]
                minibatchY = y_onehot[self.batch_size * batch:self.batch_size * (batch + 1), :]
                cache = self.forward(minibatchX)
                grads = self.backward(cache, minibatchY)
                self.update(grads)

                trainLoss += self.loss(cache[f"H{self.n_hidden+1}"], minibatchY)
                predictedY[self.batch_size * batch:self.batch_size * (batch + 1)] = np.argmax(
                    cache[f"H{self.n_hidden + 1}"], axis=1)

            X_val, y_val = self.va
            onVal_y = np.eye(np.max(y_train) - np.min(y_train) + 1)[y_val]
            valCache = self.forward(X_val)

            predicted_valY = np.argmax(valCache[f"H{self.n_hidden + 1}"], axis=1)
            valAccuracy = np.mean(y_val == predicted_valY)
            valLoss = self.loss(valCache[f"H{self.n_hidden+1}"], onVal_y)

            trAccuracy = np.mean(y_train == predictedY)

            print(f"Epoch= {epoch}, Loss={trainLoss:10.2f}, Accuracy={trAccuracy:3.6f}, Val.Loss={valLoss:10.2f}, Val.Accuracy= {valAccuracy:3.6f}")
            # break

    def test(self):
        pass

#### Test of the NN class with MNIST:

In [ ]:
neural_net = NN(datapath="mnist.pkl", hidden_dims=(500, 400))
neural_net.train("glorot")